In [1]:
# simulate input data

input_nodes = [{'id':'B1','is_start':1,'name':'data_input',
                        'in_port':[],
                        'out_port':[{'port_id':'B1_P3','port_type':'D'}]},
                
                {'id':'B2','is_start':1,'name':'data_input_2',
                         'in_port':[],
                         'out_port':[{'port_id':'B2_P3','port_type':'D'}]},
                
                {'id':'B3','is_start':0,'name':'data_process',
                         'in_port':[{'port_id':'B3_P1','port_type':'D'}],
                         'out_port':[{'port_id':'B3_P3','port_type':'D'}]},
                
                {'id':'B4','is_start':0,'name':'data_process_2',
                         'in_port':[{'port_id':'B4_P1','port_type':'D'}],
                         'out_port':[{'port_id':'B4_P3','port_type':'D'}]},
                
                {'id':'B5','is_start':0,'name':'data_merge',
                         'in_port':[{'port_id':'B5_P1','port_type':'D'},
                                    {'port_id':'B5_P2','port_type':'D'}],
                         'out_port':[{'port_id':'B5_P3','port_type':'D'}]},
                
                {'id':'B6','is_start':0,'name':'data_process_3',
                         'in_port':[{'port_id':'B6_P1','port_type':'D'}],
                         'out_port':[{'port_id':'B6_P3','port_type':'D'}]},
                
                {'id':'B7','is_start':0,'name':'data_split',
                         'in_port':[{'port_id':'B7_P1','port_type':'D'}],
                         'out_port':[{'port_id':'B7_P3','port_type':'D'},
                                     {'port_id':'B7_P4','port_type':'D'}]},
                
                {'id':'B8','is_start':0,'name':'model',
                         'in_port':[{'port_id':'B8_P1','port_type':'M'}],
                         'out_port':[{'port_id':'B8_P3','port_type':'D'}]},
                
                {'id':'B9','is_start':0,'name':'test',
                         'in_port':[{'port_id':'B9_P1','port_type':'M'},
                                    {'port_id':'B9_P2','port_type':'D'}],
                         'out_port':[{'port_id':'B9_P3','port_type':'D'}]},
                
                {'id':'B10','is_start':0,'name':'evaluation',
                         'in_port':[{'port_id':'B10_P1','port_type':'D'}],
                         'out_port':[]}]

input_lines = [{'start_port':'B1_P3','end_port':'B3_P1'},{'start_port':'B2_P3','end_port':'B4_P1'},{'start_port':'B3_P3','end_port':'B5_P1'},
               {'start_port':'B4_P3','end_port':'B5_P2'},{'start_port':'B5_P3','end_port':'B6_P1'},{'start_port':'B6_P3','end_port':'B7_P1'},
               {'start_port':'B7_P3','end_port':'B8_P1'},{'start_port':'B7_P4','end_port':'B9_P2'},{'start_port':'B8_P3','end_port':'B9_P1'},
               {'start_port':'B9_P3','end_port':'B10_P1'}]

# Node class for tree traversal
class Node(object):
    node_id =''
    node_name =''
    isbreak = False
    code_str = ''
    in_port_1,in_port_2 = None,None
    out_port_1,out_port_2 = None,None
    next_port_1,next_port_2 = None,None
    next_node_1,next_node_2 = None,None
    
    def __init__(self,node_id,node_name):
        self.node_id = node_id
        self.node_name = node_name
    
    # find the ports which Node conect to based on lines
    def NextPortFiner(self,line_dict):
        if self.out_port_1:
            self.next_port_1 = line_dict.get(self.out_port_1)
        if self.out_port_2:
            self.next_port_2 = line_dict.get(self.out_port_2)
            
    # find the Node which Node conect to based on next_port and node dict        
    def NextNodeFiner(self,node_helper):
        if self.next_port_1:
            node_id = self.next_port_1.split('_')[0]
            nextNode = node_helper.get(node_id)
            self.next_node_1 = nextNode
        if self.next_port_2:
            node_id = self.next_port_2.split('_')[0]
            nextNode = node_helper.get(node_id)
            self.next_node_2 = nextNode
            
    # generate code for each Node use port_id as input and output for now
    def GenOutPut(self):
        input_s = ''
        if self.in_port_1:input_s += self.in_port_1
        if self.in_port_2:input_s += ', ' + self.in_port_2
        ouput_s = ''
        if self.out_port_1:ouput_s += self.out_port_1
        if self.out_port_2:ouput_s += ', ' + self.out_port_2
        if self.out_port_1 or self.out_port_2:ouput_s += ' = '
        self.code_str = '\t'+ouput_s + self.node_name+'(' + input_s + ')\n' 

        
# preorder tree traversal
def preorder(node,node_helper,res,force=False):
    if node:
        if not node.node_id=='root':
            node.NextNodeFiner(node_helper)
            node.GenOutPut()
        if not node.isbreak or force:
            res.append(node.code_str)
            preorder(node.next_node_1,node_helper,res)
            preorder(node.next_node_2,node_helper,res)
    return res
        

# store line imformation
line_dict = {}
for line in input_lines:
    start = line.get('start_port')
    end = line.get('end_port')
    line_dict.update({start:end})

# vertual root node for tree traversal
root = Node('root','root')
root.code_str = "if __name__ == '__main__':\n"

# break list is the start of each tree traversal
start_list = [root]

# store node information in Node and node_helper
node_helper = {}
for node in input_nodes:
    temp = Node(node.get('id'),node.get('name'))
    node_helper.update({node.get('id'):temp})
    
    if node.get('is_start')==1:
        if not root.next_node_1:
            root.next_node_1 = temp
        else:
            root.next_node_2 = temp

    for port in node.get('in_port'):
        port_id = port.get('port_id')
        if port_id[-1]=='1':
            temp.in_port_1 = port_id
        # condition to break tree traversal and start agian: has 2 in_ports
        if port_id[-1]=='2':
            temp.in_port_2 = port_id
            start_list.append(temp)
            temp.isbreak = True
            
    for port in node.get('out_port'):
        port_id = port.get('port_id')
        if port_id[-1]=='3':
            temp.out_port_1 = port_id
        if port_id[-1]=='4':
            temp.out_port_2 = port_id
    temp.NextPortFiner(line_dict)

# tree traversal for each start node
code_list= ''
res = []
for node in start_list:
    preorder(node,node_helper,res,True)
code_list += ' '.join([str(elem) for elem in res])
    
# unique name for each output
import random
rand_list = []
for start,end in line_dict.items():
    suffix = str(random.randint(1000,9999))
    while suffix in rand_list:
        suffix = str(random.randint(1000,9999))
    rand_list.append(suffix)
    code_list = code_list.replace(str(start),'output_'+ suffix)
    code_list = code_list.replace(str(end),'output_'+ suffix)

In [2]:
print(code_list)

if __name__ == '__main__':
 	output_6135 = data_input()
 	output_7427 = data_process(output_6135)
 	output_5794 = data_input_2()
 	output_3275 = data_process_2(output_5794)
 	output_9736 = data_merge(output_7427, output_3275)
 	output_6256 = data_process_3(output_9736)
 	output_3103, output_4312 = data_split(output_6256)
 	output_1483 = model(output_3103)
 	output_2127 = test(output_1483, output_4312)
 	evaluation(output_2127)



In [3]:
code_list

"if __name__ == '__main__':\n \toutput_6135 = data_input()\n \toutput_7427 = data_process(output_6135)\n \toutput_5794 = data_input_2()\n \toutput_3275 = data_process_2(output_5794)\n \toutput_9736 = data_merge(output_7427, output_3275)\n \toutput_6256 = data_process_3(output_9736)\n \toutput_3103, output_4312 = data_split(output_6256)\n \toutput_1483 = model(output_3103)\n \toutput_2127 = test(output_1483, output_4312)\n \tevaluation(output_2127)\n"